# Experiment 01-01: Hello World

Test basic OpenCL functionality to ensure the environment is up and running.

## Web References

- [Parallel Programming with (Py)OpenCL for Fun and Profit](https://github.com/Gordonei/pyconza-october-2018/blob/master/code/PyConZA_OpenCL_Talk.ipynb)
- [pyopencl-in-action](https://github.com/oysstu/pyopencl-in-action/blob/master/ch4/double_test.py)

In [1]:
import sys
import numpy as np
import pyopencl

## List Available Devices

In [2]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Apple


## Get the testing platform

In [3]:
def get_first_platform_by_name(name : str) -> pyopencl.Platform:
    """
    Get the first platform with the given name.

    Parameters
    ----------
    name : str
        The name of the platform to get.
    """
    for platform in pyopencl.get_platforms():
        if platform.name == name:
            return platform

    raise ValueError(f"No platform named '{name}'")

In [4]:
gpu_platform = get_first_platform_by_name("Apple")
gpu_devices = gpu_platform.get_devices(device_type=pyopencl.device_type.GPU)
gpu_context = pyopencl.Context(devices=gpu_devices)

print(gpu_devices)
print(gpu_context)

[<pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>]
<pyopencl.Context at 0x600002034630 on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>>


## Build The Program

In [5]:
def build_program(program_files : list, context : pyopencl.Context,
        build_options=[]) -> pyopencl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    context : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source_code = ''

    for ocl_file in program_files:
        with open(ocl_file, 'r') as opencl_file:
            file_source_code = opencl_file.read()
            program_source_code += '\n' + file_source_code

    program_source = pyopencl.Program(context, program_source_code)
    program = program_source.build(options=build_options)
            
    return program

In [6]:
gpu_program = build_program(['ex01-01_hello_world.cl'], gpu_context)

print(gpu_program)

In [7]:
# show the kernel names
program_kernel_names = gpu_program.get_info(pyopencl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: summy


## Run Program

In [8]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (value, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=value, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size, *kernel_arguments)

    # copying data off the device
    for (value, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=value)
        
    # waiting for everything to finish
    queue.finish()

### Device Memory Setup

In [9]:
# input values
a = np.float32(256.2)
b = np.float32(25.6)
c = np.zeros(shape=(1,), dtype=np.float32)

# create the input tuples
input_tuples = (
    (a, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, sys.getsizeof(a))),
    (b, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, sys.getsizeof(b))),
)

# create the output tuples
output_tuples = (
    (c, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.WRITE_ONLY, c.itemsize)),
)

### Execute Program

In [10]:
print(gpu_program.summy)

In [11]:
global_size = (1,)
local_size = None

gpu_queue = pyopencl.CommandQueue(gpu_context)

run_ocl_kernel(gpu_queue, gpu_program.summy, global_size, input_tuples, output_tuples, local_size)
print(f"c = {c[0]}")

281.800018
c = 281.8000183105469
